In [1]:
from langgraph.graph import StateGraph, START, END
from langchain_huggingface import ChatHuggingFace, HuggingFaceEndpoint
from dotenv import load_dotenv
from typing import TypedDict

In [2]:

llm = HuggingFaceEndpoint(
    repo_id="openai/gpt-oss-120b",
    task="text-generation"
)

model = ChatHuggingFace(llm=llm)

/home/bumblebee/AgenticAI/venv/lib64/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
class BlogState(TypedDict):

    title: str
    outline: str
    content: str

In [4]:
def create_outline(state: BlogState) -> BlogState:

    # fetch title
    title = state['title']

    # call llm gen outline
    prompt = f'Generate a detailed outline for a blog on the topic - {title}'
    outline = model.invoke(prompt).content

    # update state
    state['outline'] = outline

    return state

In [5]:
def create_blog(state: BlogState) -> BlogState:

    title = state['title']
    outline = state['outline']

    prompt = f'Write a detailed blog on the title - {title} using the follwing outline \n {outline}'

    content = model.invoke(prompt).content

    state['content'] = content

    return state

In [6]:
graph = StateGraph(BlogState)

# nodes
graph.add_node('create_outline', create_outline)
graph.add_node('create_blog', create_blog)

# edges
graph.add_edge(START, 'create_outline')
graph.add_edge('create_outline', 'create_blog')
graph.add_edge('create_blog', END)

workflow = graph.compile()


In [7]:
intial_state = {'title': 'Rise of AI in India'}

final_state = workflow.invoke(intial_state)

print(final_state)

{'title': 'Rise of AI in India', 'outline': '**Blog Title:** *The Rise of AI in India – Opportunities, Challenges, and the Road Ahead*  \n\n**Target Audience:** Tech‑savvy professionals, policymakers, investors, startup founders, students, and anyone curious about how artificial intelligence is reshaping India’s economy and society.  \n\n---\n\n## 1️⃣ Introduction  \n- **Hook:** A striking statistic (e.g., “India is projected to have 190\u202fmillion AI‑enabled jobs by 2030” or “AI funding in India hit $2.5\u202fbn in FY‑2024”).  \n- **Why it matters:** Briefly connect AI to India’s demographic dividend, digital push, and global competitiveness.  \n- **Thesis statement:** Outline the blog’s purpose – to map the forces driving AI’s rapid adoption, showcase key players, discuss hurdles, and envision the next decade.  \n\n---\n\n## 2️⃣ Historical Context – From “IT Hub” to “AI Hub”  \n### 2.1 Early Foundations  \n- 1990s‑2000s: Establishment of computer science programs at IITs, IISc, and

In [8]:
print(final_state['outline'])

**Blog Title:** *The Rise of AI in India – Opportunities, Challenges, and the Road Ahead*  

**Target Audience:** Tech‑savvy professionals, policymakers, investors, startup founders, students, and anyone curious about how artificial intelligence is reshaping India’s economy and society.  

---

## 1️⃣ Introduction  
- **Hook:** A striking statistic (e.g., “India is projected to have 190 million AI‑enabled jobs by 2030” or “AI funding in India hit $2.5 bn in FY‑2024”).  
- **Why it matters:** Briefly connect AI to India’s demographic dividend, digital push, and global competitiveness.  
- **Thesis statement:** Outline the blog’s purpose – to map the forces driving AI’s rapid adoption, showcase key players, discuss hurdles, and envision the next decade.  

---

## 2️⃣ Historical Context – From “IT Hub” to “AI Hub”  
### 2.1 Early Foundations  
- 1990s‑2000s: Establishment of computer science programs at IITs, IISc, and other premier institutions.  
- Early research labs (e.g., TCS Resear

In [10]:
print(final_state['content'])

# The Rise of AI in India – Opportunities, Challenges, and the Road Ahead  

*Target audience: tech‑savvy professionals, policymakers, investors, startup founders, students, and anyone curious about how artificial intelligence is reshaping India’s economy and society.*

---

## 1️⃣ Introduction  

### Hook  
> **“India is projected to generate **190 million AI‑enabled jobs** by 2030, and AI‑focused venture capital funding crossed **$2.5 bn** in FY‑2024 alone.”**  

These numbers aren’t just headlines – they signal a tectonic shift in the way India creates value, solves problems, and competes on the global stage.

### Why it matters  
India sits on a **demographic dividend** of more than **1.4 bn people**, the world’s youngest workforce, and a **digital push** powered by **Digital India**, **5G**, and a burgeoning startup ecosystem. Harnessing AI can amplify productivity, bridge rural‑urban gaps, and propel the country into the top tier of **AI‑driven economies**.

### Thesis statement 